<a href="https://colab.research.google.com/github/muneeb706/machine-learning/blob/main/knn_nb_pca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sklearn

In [2]:
import os
import sys

import numpy as np
import pandas as pd

from sklearn.model_selection import LeaveOneOut

from sklearn.neighbors import KNeighborsClassifier

# Problem 1: KNN Classifier

You are asked to build a k-Nearest Neighbor (kNN) classifier. The data set for evaluation is the
heart data set. More information about the data can be found here https://archive.ics.uci.
edu/ml/datasets/Statlog+(Heart). The data set is included in the provided data folder. In
the heart data folder, there are three files: \trainSet.txt", \trainLabels.txt", and \test.txt". Each
row of \trainSet.txt" corresponds to a data point whose class label is provided in the same row of
\trainLabels.txt". Each row of \testSet.txt" corresponds to a data point whose class label needs
to be predicted. You will train a classification model using \trainSet.txt" and \trainLabels.txt",
and use it to predict the class labels for the data points in \testSet.txt".
Use the leave one out cross validation on the training data to select the best k among f1; 2; : : : ; 10g.
Report the averaged leave-one-out error (averaged over all training data points) for each k 2
f1; 2; : : : ; 10g and the best k used for predicting the class labels for test instances. You should also
report the predicted labels for the testSet.

Loading the heart dataset, downloaded from (https://archive.ics.uci.edu/ml/datasets/Statlog+(Heart))

In [3]:
train_data = pd.read_csv('./sample_data/heart_trainSet.txt', sep=",", header=None)
train_data.columns = ["column-1", "column-2", "column-3", "column-4", 
                            "column-5", "column-6", "column-7", "column-8", "column-9", 
                            "column-10", "column-11", "column-12", "column-13"]
train_data.head()

,column-1,column-2,column-3,column-4,column-5,column-6,column-7,column-8,column-9,column-10,column-11,column-12,column-13
0,0.041667,1,1.00000,-0.47170,-0.26941,-1,1,-0.312980,1,0.032258,0,0.33333,-1.0
1,0.208330,1,1.00000,-0.88679,-0.50685,-1,-1,0.297710,-1,-0.967740,-1,-0.33333,1.0
2,0.583330,1,1.00000,-0.88679,-0.21005,-1,1,-0.175570,1,-0.709680,0,0.33333,-1.0
3,0.000000,-1,1.00000,-0.32076,-0.36986,-1,1,0.099237,-1,-0.870970,0,-1.00000,-1.0
4,0.166670,1,0.33333,-0.35849,-0.52968,-1,1,0.206110,-1,-0.870970,0,-0.33333,1.0


In [4]:
train_data_labels = pd.read_csv('./sample_data/heart_trainLabels.txt', sep=",", header=None)
train_data_labels.columns = ["target"]
train_data_labels.head()

,target
0,1
1,1
2,1
3,-1
4,1


In [5]:
test_data = pd.read_csv('./sample_data/heart_testSet.txt', sep=",", header=None)
test_data.columns = ["column-1", "column-2", "column-3", "column-4", 
                            "column-5", "column-6", "column-7", "column-8", "column-9", 
                            "column-10", "column-11", "column-12", "column-13"]
test_data.head()

,column-1,column-2,column-3,column-4,column-5,column-6,column-7,column-8,column-9,column-10,column-11,column-12,column-13
0,0.291670,-1,1.00000,0.20755,-0.18265,-1,1,0.374050,-1,-1.000000,-1,-1.00000,-1.0
1,0.166670,1,-0.33333,-0.43396,-0.38356,-1,-1,0.068702,-1,-0.903230,-1,-1.00000,1.0
2,-0.291670,1,1.00000,-0.13207,-0.15525,-1,-1,-0.251910,1,-0.419350,0,0.33333,1.0
3,-0.500000,1,0.33333,-0.32076,-0.59817,-1,1,0.480920,-1,-0.354840,0,-1.00000,-1.0
4,0.041667,1,1.00000,-0.69811,-0.48402,-1,-1,-0.160310,1,-0.096774,0,-0.33333,1.0


Leave One Out Cross Validation for each k = [1,2,.....10]

In [6]:
ks = [1,2,3,4,5,6,7,8,9,10]
loo = LeaveOneOut()
mean_errors = []
for k in ks:
  neigh = KNeighborsClassifier(n_neighbors=k)
  errors = []
  for train_index, test_index in loo.split(train_data):
    X_train, X_test = train_data.iloc[train_index,0:], train_data.iloc[test_index,0:]
    y_train, y_test = train_data_labels.iloc[train_index,0], train_data_labels.iloc[test_index,0]
    neigh.fit(X_train, y_train)
    accuracy = neigh.score(X_test, y_test)
    errors.append(1 - accuracy)

  mean_errors.append(np.mean(errors))
  
min_error_index = mean_errors.index(min(mean_errors))
best_k = ks[min_error_index]

print("Leave One out errors: ", mean_errors)
print("Best K: ", best_k)
print("Corresponding leave one out error: ", min(mean_errors))

Leave One out errors:  [0.22633744855967078, 0.22633744855967078, 0.1728395061728395, 0.18518518518518517, 0.18106995884773663, 0.1646090534979424, 0.16872427983539096, 0.18106995884773663, 0.18106995884773663, 0.17695473251028807]
Best K:  6
Corresponding leave one out error:  0.1646090534979424


Predicitng labels for the test using best value of k obtained in previous step.

In [7]:
 best_neigh = KNeighborsClassifier(n_neighbors=best_k)
 best_neigh.fit(train_data.iloc[:,:], train_data_labels.iloc[:,0])
 best_neigh.predict(test_data.iloc[:,:])

array([-1, -1,  1, -1,  1, -1, -1,  1, -1,  1,  1,  1, -1, -1, -1,  1,  1,
       -1, -1, -1,  1,  1,  1,  1, -1, -1, -1])

# Problem 2 (b): Naive Bayes for Text Classification

The data set for training and evaluation is the 20NewsGroup data, which is included in
the provided data folder. You will find six text files in this data set: train.data, train.label,
train.map, test.data, test.label, and test.map, where the first three files are used for training
and the last three files are for testing. In the train.data file, you will find the word histograms
of all documents; each row is a tuple of format (document-id, word-id, word-occurrence). The
class labels of training documents can be found in train.label with the order corresponding to
training documents' id, and the topic of each class can be found in train.map. Similarly, the
word histograms and the class assignments of test documents can be found in test.data and
test.label, respectively. In this problem, you need to train a Naive Bayes classifier with the
Laplace smoothing using the training data and apply the learned classifier to predict the class
labels for the test documents.

In [8]:
import numpy as np
import pandas as pd

Loading 20 News Group Dataset.

In [9]:
ng_train_data = pd.read_csv('./sample_data/train.data', sep=" ", header=None)
ng_train_data.columns = ["document-id", "word-id", "word-occurrence"]
ng_train_data.head()

,document-id,word-id,word-occurrence
0,1,1,4
1,1,2,2
2,1,3,10
3,1,4,4
4,1,5,2


In [10]:
ng_train_labels = pd.read_csv('./sample_data/train.label', sep=" ", header=None)
ng_train_labels.columns = ["target"]
ng_train_labels.head()

,target
0,1
1,1
2,1
3,1
4,1


In [11]:
ng_labels_map = pd.read_csv('./sample_data/train.map', sep=" ", header=None)
ng_labels_map.columns = ["category-name", "category-id"]
ng_labels_map.head()

,category-name,category-id
0,alt.atheism,1
1,comp.graphics,2
2,comp.os.ms-windows.misc,3
3,comp.sys.ibm.pc.hardware,4
4,comp.sys.mac.hardware,5


In [14]:
ng_test_data = pd.read_csv('./sample_data/test.data', sep=" ", header=None)
ng_test_data.columns = ["document-id", "word-id", "word-occurrence"]
ng_test_data.head()

,document-id,word-id,word-occurrence
0,1,3,1
1,1,10,1
2,1,12,8
3,1,17,1
4,1,23,8


In [15]:
ng_test_labels = pd.read_csv('./sample_data/test.label', sep=" ", header=None)
ng_test_labels.columns = ["target"]
ng_test_labels.head()

,target
0,1
1,1
2,1
3,1
4,1


Generating documents based on data.

In [16]:
#key = docid, value="document text"
train_docs = {}
words = []
for index, row in ng_train_data.iterrows():
    doc_id = row['document-id']
    word_id = str(row['word-id'])
    word_freq = row['word-occurrence']
    if doc_id not in train_docs.keys():
      train_docs[doc_id] = ""
    
    if word_id not in words:
        words.append(word_id)
      
    for i in range(0,word_freq):
      word =  word_id + " "
      train_docs[doc_id] += word

print("Nmber of unique words: ", len(words))
print("Number of Training Documents: ", (len(train_docs.keys())))


Nmber of unique words:  53975
Number of Training Documents:  11269


In [17]:
#key = docid, value="document text"
test_docs = {}
for index, row in ng_test_data.iterrows():
    doc_id = row['document-id']
    word_id = str(row['word-id'])
    word_freq = row['word-occurrence']
    if doc_id not in test_docs.keys():
      test_docs[doc_id] = ""
    
    for i in range(0,word_freq):
      word =  word_id + " "
      test_docs[doc_id] += word

print("Number of Testing Documents: ", (len(test_docs.keys())))

Number of Testing Documents:  7505


Extracting TF-IDF vectors.

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

# removing stop words as we are dealing with word ids, we don't know which ones are stop words
vectorizer = TfidfVectorizer(stop_words = None, min_df=0)
train_vectors = vectorizer.fit_transform(train_docs.values())
train_vectors.shape

(11269, 53966)

In [19]:
test_vectors = vectorizer.transform(test_docs.values())
test_vectors.shape

(7505, 53966)

Training and predicting using Naive Bayes Classifier

Calculating Classification Accuracy.

In [20]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB(alpha=.01)
clf.fit(train_vectors, ng_train_labels['target'].values)
preds = clf.predict(test_vectors)

Mapping labels with their names and exporting them to text files.

In [21]:
predictions = []
for pred in preds:
  category_name = ng_labels_map[ng_labels_map['category-id'] == pred]['category-name']
  predictions.append(str(pred)+" : "+ category_name.values[0])

with open('category_predictions.txt', 'w') as f:
    for prediction in predictions:
        f.write("%s\n" % prediction)

In [22]:
test_labels = ng_test_labels["target"].values
index=0
misclassified = 0
for pred in preds:
  if pred != test_labels[index]:
    misclassified+=1
  index+=1

accuracy = 1-(misclassified/len(test_labels))
print("Classification Accuracy: ", accuracy*100)

Classification Accuracy:  81.83877415056628


# Problem 3: PCA

You are asked to build a k-Nearest Neighbor (kNN) classifier based on dimenationality reduced
data by PCA. The data set for evaluation is the gisette data set. More information about the
data can be found here https://archive.ics.uci.edu/ml/datasets/Gisette. The data set is
included in the provided data folder. The data is in the same format as that in Problem 1. You will
train a classification model using \trainSet.txt" and \trainLabels.txt", and use it to predict the
class labels for the data points in \testSet.txt". First, train a kNN based on the original features
and then conduct PCA on the data and learn a kNN model using the reduced data. Report the
performance of both models on the testing data. For learning both models, you should also conduct
cross-validation (of your choice) to select the best k. Describe the cross-validation approach and
report the best value of k for both models.

In [5]:
import numpy as np
import pandas as pd

Loading datasets.

In [6]:
gisette_train_data = pd.read_csv('./sample_data/gisette_trainSet.txt', sep=" ", header=None)
gisette_train_data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,4960,4961,4962,4963,4964,4965,4966,4967,4968,4969,4970,4971,4972,4973,4974,4975,4976,4977,4978,4979,4980,4981,4982,4983,4984,4985,4986,4987,4988,4989,4990,4991,4992,4993,4994,4995,4996,4997,4998,4999
0,0.1011,-1.0,-0.009009,-1.0,-1.0,-1.0,-1.0,0.95395,-1.0,-1.00000,-1.0,-1.0,0.96797,-1.0,0.99199,-1.00000,0.96797,-1.0,-1,0.96797,-1.0,-1,-1.0,-1.000000,-1.0,0.96797,-1.0,-1.00000,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.00000,-1.0000,-1.0,-1.0,0.98398,0.96797,...,-1.0,-1.0,-1.0,0.027027,0,0.067067,0.62362,-1.00000,-1.00000,-1.0,0.96797,-1.00000,-1.0,-1.0,-1.0,-1.0,0.96797,0.98398,-1.00000,-1.00000,-1.0,0.96797,-1.0,-1.00000,-1.00000,-1.0,-1.0,-1.0,-1.0,-1.00000,-1.00000,-1.000000,-1.0,0.98398,0.98398,-1.0,-1.00000,-1.0,-1.0,0.96797
1,-1.0000,-1.0,-1.000000,-1.0,-1.0,-1.0,-1.0,0.95395,-1.0,-1.00000,-1.0,-1.0,-1.00000,-1.0,0.16917,-1.00000,-1.00000,-1.0,-1,-1.00000,-1.0,-1,-1.0,-1.000000,-1.0,0.96797,-1.0,-1.00000,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.00000,0.7978,-1.0,-1.0,-1.00000,-1.00000,...,-1.0,-1.0,-1.0,0.865870,0,-1.000000,0.52953,0.65766,-1.00000,-1.0,-1.00000,0.96797,-1.0,-1.0,-1.0,-1.0,-1.00000,0.98398,-1.00000,-1.00000,-1.0,-1.00000,-1.0,-1.00000,-1.00000,-1.0,-1.0,-1.0,-1.0,-1.00000,-1.00000,-0.049049,-1.0,0.98398,-1.00000,-1.0,0.98398,-1.0,-1.0,-1.00000
2,-1.0000,-1.0,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.00000,-1.0,-1.00000,-1.0,-1.0,0.96797,-1.0,0.99199,0.96797,0.95395,-1.0,-1,-1.00000,-1.0,-1,-1.0,-1.000000,-1.0,0.96797,-1.0,0.48548,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.00000,-1.0000,-1.0,-1.0,-1.00000,-1.00000,...,-1.0,-1.0,-1.0,0.983980,0,-1.000000,0.98398,-1.00000,-1.00000,-1.0,-1.00000,-1.00000,-1.0,-1.0,-1.0,-1.0,-1.00000,0.16917,-1.00000,-1.00000,-1.0,-1.00000,-1.0,0.88188,-1.00000,-1.0,-1.0,-1.0,-1.0,0.23323,-1.00000,-1.000000,-1.0,-1.00000,-1.00000,-1.0,-1.00000,-1.0,-1.0,-1.00000
3,-1.0000,-1.0,0.485480,-1.0,-1.0,-1.0,-1.0,0.36937,-1.0,0.91391,-1.0,-1.0,0.96797,-1.0,0.98398,0.63363,0.96797,-1.0,-1,-1.00000,-1.0,-1,-1.0,-1.000000,-1.0,0.37137,-1.0,0.53554,-1,-1.0,-1.0,-1.0,-1.0,-1.0,0.98398,-1.0000,-1.0,-1.0,-1.00000,-1.00000,...,-1.0,-1.0,-1.0,0.497500,0,-1.000000,0.45946,-1.00000,0.98398,-1.0,0.61762,-1.00000,-1.0,-1.0,-1.0,-1.0,-1.00000,0.98398,-1.00000,0.74975,-1.0,0.71972,-1.0,-1.00000,-1.00000,-1.0,-1.0,-1.0,-1.0,-1.00000,-1.00000,-1.000000,-1.0,-1.00000,-1.00000,-1.0,0.34935,-1.0,-1.0,0.67768
4,-1.0000,-1.0,-1.000000,-1.0,-1.0,-1.0,-1.0,0.21722,-1.0,0.95996,-1.0,-1.0,-1.00000,-1.0,0.94595,-1.00000,-1.00000,-1.0,-1,-1.00000,-1.0,-1,-1.0,-0.039039,-1.0,-1.00000,-1.0,-1.00000,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.00000,-1.0000,-1.0,-1.0,0.98398,-1.00000,...,-1.0,-1.0,-1.0,-1.000000,0,-1.000000,-1.00000,-1.00000,-1.00000,-1.0,-1.00000,-1.00000,-1.0,-1.0,-1.0,-1.0,-1.00000,-1.00000,0.35135,0.56156,-1.0,-1.00000,-1.0,-1.00000,0.96797,-1.0,-1.0,-1.0,-1.0,-1.00000,0.98398,-1.000000,-1.0,0.65766,-1.00000,-1.0,-1.00000,-1.0,-1.0,-1.00000


In [7]:
gisette_train_labels = pd.read_csv('./sample_data/gisette_trainLabels.txt', sep=",", header=None)
gisette_train_labels.columns = ["target"]
gisette_train_labels.head()

,target
0,1
1,-1
2,1
3,1
4,1


In [8]:
gisette_test_data = pd.read_csv('./sample_data/gisette_testSet.txt', sep=" ", header=None)
gisette_test_data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,4960,4961,4962,4963,4964,4965,4966,4967,4968,4969,4970,4971,4972,4973,4974,4975,4976,4977,4978,4979,4980,4981,4982,4983,4984,4985,4986,4987,4988,4989,4990,4991,4992,4993,4994,4995,4996,4997,4998,4999
0,0.377380,-1.00000,-1.00000,-1.0,-1.0,-1.00000,-1.00000,0.90591,-1.0,0.74174,-1.00000,-1.0,-1.0,-1.000000,1.00000,-1.0000,-1.0,-1.0,-1,-1.0,-1.000000,-1,-1.000000,-1.00000,-1.0,0.95395,-1.0,-1.0,-1.0,-1,-1.0,-1.0,-1.00000,-1.0,-1.00000,-1.0,-1.0,-1.0,-1.00000,0.17517,...,-1.00000,-1.0,-1.0,0.44344,0,-1.00000,0.99199,0.92993,-1.00000,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.00000,-1.00000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.95395,-1.0000,0.27127,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.0,-0.011011,-1.00000,0.53954,-1.0
1,0.557560,0.51752,-1.00000,-1.0,-1.0,-1.00000,-1.00000,0.41742,-1.0,0.98398,0.71972,-1.0,-1.0,-1.000000,0.98398,-1.0000,-1.0,-1.0,-1,-1.0,-1.000000,-1,-1.000000,-1.00000,-1.0,-1.00000,-1.0,-1.0,-1.0,-1,-1.0,-1.0,0.32532,-1.0,-1.00000,-1.0,-1.0,-1.0,-1.00000,-1.00000,...,-1.00000,-1.0,-1.0,-1.00000,0,-1.00000,-1.00000,-1.00000,-1.00000,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.98398,-1.00000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.85185,0.7037,-1.00000,-1.0,-1.0,-1.0,0.541540,-1.0,-1.0,-1.000000,-1.00000,-1.00000,-1.0
2,-0.061061,-1.00000,0.63363,-1.0,-1.0,-1.00000,-1.00000,-1.00000,-1.0,-1.00000,-1.00000,-1.0,-1.0,-1.000000,0.94595,0.7978,-1.0,-1.0,-1,-1.0,-1.000000,-1,0.073073,-1.00000,-1.0,0.59560,-1.0,-1.0,-1.0,-1,-1.0,-1.0,-1.00000,-1.0,0.12112,-1.0,-1.0,-1.0,0.45946,0.26326,...,0.99196,-1.0,-1.0,0.64765,0,-1.00000,0.98398,-1.00000,0.97598,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.00000,0.40941,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.30530,-1.0000,-1.00000,-1.0,-1.0,-1.0,-0.065065,-1.0,-1.0,-1.000000,-0.16517,-1.00000,-1.0
3,-1.000000,-1.00000,-1.00000,-1.0,-1.0,0.14314,0.98398,0.96797,-1.0,0.96797,-1.00000,-1.0,-1.0,0.703700,-1.00000,0.3013,-1.0,-1.0,-1,-1.0,0.069069,-1,0.953950,-1.00000,-1.0,-1.00000,-1.0,-1.0,-1.0,-1,-1.0,-1.0,-1.00000,-1.0,-1.00000,-1.0,-1.0,-1.0,0.10511,-1.00000,...,-1.00000,-1.0,-1.0,-1.00000,0,0.62763,-1.00000,-1.00000,-1.00000,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.00000,-1.00000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.00000,-1.0000,-1.00000,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.0,-1.000000,-1.00000,-1.00000,-1.0
4,-1.000000,-1.00000,-1.00000,-1.0,-1.0,-1.00000,-1.00000,0.89990,-1.0,0.98398,-1.00000,-1.0,-1.0,0.067067,-1.00000,-1.0000,-1.0,-1.0,-1,-1.0,0.983980,-1,-1.000000,0.47347,-1.0,-1.00000,-1.0,-1.0,-1.0,-1,-1.0,-1.0,-1.00000,-1.0,-1.00000,-1.0,-1.0,-1.0,-1.00000,0.95395,...,-1.00000,-1.0,-1.0,-1.00000,0,0.72773,-1.00000,-1.00000,0.41942,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.00000,-1.00000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.94595,-1.0000,-1.00000,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.0,-1.000000,-1.00000,0.95395,-1.0


In [9]:
gisette_test_labels = pd.read_csv('./sample_data/gisette_testLabels.txt', sep=",", header=None)
gisette_test_labels.columns = ["target"]
gisette_test_labels.head()

,target
0,1
1,1
2,-1
3,1
4,1


Training KNN Classifier without feature selection.

In [10]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold

cross_validation = KFold(n_splits=5, random_state=10, shuffle=True)

ks = [1,2,3,4,5,6,7,8,9,10]
mean_accs = []
for k in ks:
  neigh = KNeighborsClassifier(n_neighbors=k)
  accs = []    
  for train_index, test_index in cross_validation.split(gisette_train_data):
    # separating test data from training data based on return indices of training and testing sets
    x_train, x_test = gisette_train_data.iloc[train_index,0:],gisette_train_data.iloc[test_index,0:],
    y_train, y_test = gisette_train_labels.iloc[train_index,0],gisette_train_labels.iloc[test_index,0]

    neigh.fit(x_train, y_train)
    accuracy = neigh.score(x_test, y_test)
    accs.append(accuracy)

  mean_accs.append(np.mean(accs))
print("Mean Accuracies w.r.t different values of k, without feature selection:")   
mean_accs

Mean Accuracies w.r.t different values of k, without feature selection:


[0.9568333333333333,
 0.9601666666666666,
 0.9631666666666666,
 0.966,
 0.9606666666666666,
 0.9628333333333334,
 0.9605,
 0.9629999999999999,
 0.9611666666666666,
 0.9623333333333335]

Selecting best k.

In [11]:
max_acc_index = mean_accs.index(max(mean_accs))
best_k = ks[max_acc_index]

print("Best K, without feature selection: ", best_k)
print("Corresponding Accuracy: ", max(mean_accs))

Best K, without feature selection:  4
Corresponding Accuracy:  0.966


Training KNN Classifier witht feature selection using PCA.

In [14]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

pca = make_pipeline(StandardScaler(),
                    PCA())

ks = [1,2,3,4,5,6,7,8,9,10]
mean_accs_pca = []
for k in ks:
  neigh = KNeighborsClassifier(n_neighbors=k)
  accs = []    
  for train_index, test_index in cross_validation.split(gisette_train_data):
    # separating test data from training data based on return indices of training and testing sets
    x_train, x_test = gisette_train_data.iloc[train_index,0:],gisette_train_data.iloc[test_index,0:],
    y_train, y_test = gisette_train_labels.iloc[train_index,0],gisette_train_labels.iloc[test_index,0]

    pca.fit(x_train, y_train)
    neigh.fit(pca.transform(x_train), y_train)
    accuracy = neigh.score(pca.transform(x_test), y_test)
    accs.append(accuracy)

  mean_accs_pca.append(np.mean(accs))
print("Mean Accuracies w.r.t different values of k, with feature selection (PCA):")   
mean_accs_pca

Mean Accuracies w.r.t different values of k, with feature selection (PCA):


[0.9231666666666666,
 0.9101666666666667,
 0.9445,
 0.9366666666666665,
 0.9501666666666667,
 0.9441666666666666,
 0.9553333333333333,
 0.9463333333333332,
 0.9528333333333334,
 0.9443333333333334]

In [15]:
max_acc_index_pca = mean_accs_pca.index(max(mean_accs_pca))
best_k_pca = ks[max_acc_index_pca]

print("Best K, with feature selection (PCA): ", best_k_pca)
print("Corresponding Accuracy: ", max(mean_accs_pca))

Best K, with feature selection:  7
Corresponding Accuracy:  0.9553333333333333


## Cross validation explanation:

For this purpose, I have used five fold cross validation methods and in case of PCA. I have applied pca in every step of closs validation in other words, I have applied PCA in every cross validated treaining and testing set. With this approach we get better generlization error.